In [3]:
import gradio as gr
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# --- Load and train Diabetes model ---
diabetes_data = pd.read_csv("diabetes.csv")
diabetes_features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                     'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X_diabetes = diabetes_data[diabetes_features]
y_diabetes = diabetes_data['Outcome']
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_diabetes, y_diabetes, test_size=0.2, random_state=42)
diabetes_model = GaussianNB()
diabetes_model.fit(X_train_d, y_train_d)
diabetes_acc = accuracy_score(y_test_d, diabetes_model.predict(X_test_d))

# --- Load and train Heart Disease model ---
heart_data = pd.read_csv("heart.csv")
heart_data = pd.get_dummies(heart_data, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], drop_first=True)
X_heart = heart_data.drop('HeartDisease', axis=1)
y_heart = heart_data['HeartDisease']
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_heart, y_heart, test_size=0.2, random_state=42)
heart_model = GaussianNB()
heart_model.fit(X_train_h, y_train_h)
heart_acc = accuracy_score(y_test_h, heart_model.predict(X_test_h))

# --- Helper function for animated plot ---
def create_animated_bar(probabilities, classes, title):
    # Generate frames with incremental probability fill (simulate animation)
    frames = []
    n_frames = 20
    for i in range(1, n_frames + 1):
        frame_probs = probabilities * (i / n_frames)
        df = pd.DataFrame({'Class': classes, 'Probability': frame_probs})
        frames.append(df)

    # Combine all frames into one DataFrame with a frame id for animation
    all_frames = pd.concat([df.assign(Frame=i) for i, df in enumerate(frames)], ignore_index=True)

    fig = px.bar(all_frames, x='Class', y='Probability', color='Class',
                 animation_frame='Frame',
                 range_y=[0,1],
                 color_discrete_map={'Not Diabetic': '#6cba6b', 'Diabetic': '#e76f51',
                                     'No Disease': '#6cba6b', 'Heart Disease': '#e76f51'},
                 title=title)

    fig.update_layout(yaxis_title="Probability", showlegend=False)
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100  # Speed up animation
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 100
    return fig

# --- Prediction Functions ---
def predict_diabetes(*inputs):
    input_data = [float(i) for i in inputs]
    pred_proba = diabetes_model.predict_proba([input_data])[0]
    prediction = diabetes_model.predict([input_data])[0]
    classes = ['Not Diabetic', 'Diabetic']
    result = "🟥 Diabetic" if prediction == 1 else "🟩 Not Diabetic"
    fig = create_animated_bar(pred_proba, classes, "Diabetes Prediction Confidence")
    return result, f"{diabetes_acc * 100:.2f}%", fig

def predict_heart(*inputs):
    input_data = [float(i) for i in inputs]
    pred_proba = heart_model.predict_proba([input_data])[0]
    prediction = heart_model.predict([input_data])[0]
    classes = ['No Disease', 'Heart Disease']
    result = "❤️ Heart Disease Detected" if prediction == 1 else "💚 No Heart Disease"
    fig = create_animated_bar(pred_proba, classes, "Heart Disease Prediction Confidence")
    return result, f"{heart_acc * 100:.2f}%", fig

# --- UI Inputs ---
diabetes_inputs = [gr.Number(label=f"{feature}", value=float(X_diabetes[feature].mean()),
                            info=f"Enter patient's {feature}") for feature in diabetes_features]

heart_inputs = [gr.Number(label=col, value=float(X_heart[col].mean()),
                         info=f"Feature: {col}") for col in X_heart.columns]

# --- Build Gradio Interfaces ---
with gr.Blocks(css="""
    body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background: #f9fafb;}
    h1 { color: #3b82f6; }
    .output-text { font-size: 18px; font-weight: bold; margin: 10px 0; }
""") as app:
    gr.Markdown("<h1 style='text-align:center'>🧬 Health Disease Predictor</h1>")

    tabs = gr.Tabs()
    with tabs:
        with gr.TabItem("🩺 Diabetes"):
            with gr.Row():
                with gr.Column(scale=1):
                    diabetes_inputs_ui = [gr.Number(label=feature, value=float(X_diabetes[feature].mean()), interactive=True) for feature in diabetes_features]
                    predict_btn_d = gr.Button("Predict Diabetes", variant="primary")
                with gr.Column(scale=1):
                    output_d = gr.Textbox(label="Prediction Result", interactive=False)
                    accuracy_d = gr.Textbox(label="Model Accuracy", interactive=False)
                    plot_d = gr.Plot()

            def diabetes_handler(*args):
                return predict_diabetes(*args)
            predict_btn_d.click(fn=diabetes_handler, inputs=diabetes_inputs_ui, outputs=[output_d, accuracy_d, plot_d])

        with gr.TabItem("❤️ Heart Disease"):
            with gr.Row():
                with gr.Column(scale=1):
                    heart_inputs_ui = [gr.Number(label=col, value=float(X_heart[col].mean()), interactive=True) for col in X_heart.columns]
                    predict_btn_h = gr.Button("Predict Heart Disease", variant="primary")
                with gr.Column(scale=1):
                    output_h = gr.Textbox(label="Prediction Result", interactive=False)
                    accuracy_h = gr.Textbox(label="Model Accuracy", interactive=False)
                    plot_h = gr.Plot()

            def heart_handler(*args):
                return predict_heart(*args)
            predict_btn_h.click(fn=heart_handler, inputs=heart_inputs_ui, outputs=[output_h, accuracy_h, plot_h])

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0cc24d1a8226d5539e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
